In [52]:
!pip3 install opencv-python

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [58]:
!pip install colour

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
!pip install webcolors

In [19]:
import cv2
import os
import pandas as pd
import numpy as np
import webcolors
from colour import Color

from PIL import Image
import requests

from transformers import CLIPProcessor, CLIPModel


In [20]:
pdf_directory = os.listdir("C:\Thesis\Images")

print(pdf_directory)

['3M_2016', 'AaltoUniversity_2015']


In [21]:
# Define a function to extract green color percentage image
def extract_green_colors(image_path):
    image = cv2.imread(image_path)
    
    #Convert the image to the HSV color space
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    # Define the range of green color
    lower_range = np.array([36,25,25])
    upper_range = np.array([86, 255,255])
    
    # Create a mask for the green colors in images
    mask = cv2.inRange(hsv, lower_range, upper_range)
    
    # Calculate the percentage of pixels in the mask
    num_pixels = mask.shape[0] * mask.shape[1]
    num_nonzero_pixels = cv2.countNonZero(mask)
    percentage = num_nonzero_pixels*100 / num_pixels
    return percentage

In [22]:
# Define a function to extract blue color percentage image
def extract_blue_colors(image_path):
    image = cv2.imread(image_path)
    
    #Convert the image to the HSV color space
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    # Define the range of blue color
    lower_range = np.array([100, 50, 50])
    upper_range = np.array([130, 255, 255])
    
    # Create a mask for the blue colors in images
    mask = cv2.inRange(hsv, lower_range, upper_range)
    
    # Calculate the percentage of pixels in the mask
    num_pixels = mask.shape[0] * mask.shape[1]
    num_nonzero_pixels = cv2.countNonZero(mask)
    percentage = num_nonzero_pixels*100 / num_pixels
    return percentage

In [23]:
# Define a function to extract red color percentage image
def extract_red_colors(image_path):
    image = cv2.imread(image_path)
    
    #Convert the image to the HSV color space
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    # Define the range of blue color
    lower_range = np.array([0, 100, 100])  # Lower range for red color in HSV
    upper_range = np.array([10, 255, 255])  # Upper range for red color in HSV

    #lower_red_2 = np.array([160, 100, 100])  # Lower range for red color in HSV (due to wrapping)
    #upper_red_2 = np.array([180, 255, 255])  # Upper range for red color in HSV (due to wrapping)
    
    # Combining the two ranges for red color
    #lower_range = np.vstack([lower_red_1, lower_red_2])
    #upper_range = np.vstack([upper_red_1, upper_red_2])
    
    # Create a mask for the red colors in images
    mask = cv2.inRange(hsv, lower_range, upper_range)
    
    # Calculate the percentage of red pixels in the mask
    num_pixels = mask.shape[0] * mask.shape[1]
    num_nonzero_pixels = cv2.countNonZero(mask)
    percentage = num_nonzero_pixels*100 / num_pixels
    return percentage

In [24]:
# Define a function to detect live obj in an image
def detect_human(img_path):
    img = cv2.imread(img_path)
    # Load the human detection model
    human_cascade = cv2.CascadeClassifier(cv2.data.haarcascades +'haarcascade_frontalface_default.xml')
    # Detect humans in the image
    humans = human_cascade.detectMultiScale(img)
    # Return True if humans are present, False otherwise
    if len(humans) > 0:
        return True
    else:
        return False

In [ ]:
def find_dominant_color(image_path, k=1):
    # Read the image
    image = cv2.imread(image_path)

    # Reshape the image to a 2D array of pixels
    pixels = image.reshape((-1, 3))

    # Convert the pixel values to float
    pixels = np.float32(pixels)

    # Define criteria and apply kmeans()
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.2)
    _, _, centers = cv2.kmeans(pixels, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)

    # Convert the color values from float to integer
    centers = np.uint8(centers)

    # Get the dominant color
    dominant_color = centers[0]

    return dominant_color

#print(f"Dominant color (BGR): {dominant_color}")


In [65]:
# Read the image
img_path = os.path.join("C:\\Thesis\\Images", "3M_2016", "page_1_image_1.PNG")
image = cv2.imread(img_path)

# Reshape the image to a 2D array of pixels
pixels = image.reshape((-1, 3))

# Convert the pixel values to float
pixels = np.float32(pixels)

# Define the number of clusters (dominant colors) you want to find
k = 3

# Define criteria and apply kmeans()
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.2)
_, labels, centers = cv2.kmeans(pixels, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)

# Convert the color values from float to integer
centers = np.uint8(centers)

# Get the dominant colors
dominant_colors = centers.tolist()

# Categorize dominant colors
color_categories = [categorize_color(color) for color in dominant_colors]

print("Color Categories:", color_categories)


Color Categories: ['Unknown', 'Unknown', 'Unknown']


In [66]:
# Detent human objects using YOLO
def detect_human_yolo(img_path):
    # Load YOLO model and weights
    net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
    classes = []

    with open("coco.names", "r") as f:
        classes = [line.strip() for line in f.readlines()]

    # Read the image
    img = cv2.imread(img_path)
    height, width, _ = img.shape

    # Create a blob from the image
    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)

    # Set the input to the network
    net.setInput(blob)

    # Get output layer names
    output_layers = net.getUnconnectedOutLayersNames()

    # Run forward pass
    outs = net.forward(output_layers)

    # Process the output
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5 and class_id == 0:  # 0 corresponds to the 'person' class
                return True

    return False


In [43]:
def image_object_classification(img_path):
    model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14")
    processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14")

    # Load and preprocess the image
    image = Image.open(img_path)
    #inputs = processor(text=["a photo of human", "a photo of Animal", "a photo of Nature", "a photo of Vehicle or machine", "a photo of buildings"], images=image, return_tensors="pt", padding=True)
    # Use list comprehension to process each image and get a list of probabilities
    probabilities_list = [model(**processor(text=["a photo of human", "a photo of Animal", "a photo of Nature", "a photo of Vehicle or machine", "a photo of buildings"], images=image, return_tensors="pt", padding=True)).logits_per_image.softmax(dim=1).detach().numpy().tolist()]
    #outputs = model(**inputs)
    #logits_per_image = outputs.logits_per_image # this is the image-text similarity score
    #probs = logits_per_image.softmax(dim=1) # we can take the softmax to get the label probabilities
    
    #return probs
    return probabilities_list

In [53]:
img_path = os.path.join("C:\\Thesis\\Images", "3M_2016", "page_1_image_1.PNG")
probs_list = image_object_classification(img_path)
probabilities = probs_list[0]
probabilities = probabilities[0]

print(len(probabilities))
print(probabilities)

is_human = probabilities[0]
is_animal = probabilities[1]
is_nature = probabilities[2]
is_vehicle_or_machine = probabilities[3]
is_building = probabilities[4]

print(is_human)
print(is_animal)


`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["bos_token_id"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["eos_token_id"]` will be overriden.


5
[0.46359506249427795, 0.014667787589132786, 0.43232956528663635, 0.052379947155714035, 0.03702764958143234]
0.46359506249427795
0.014667787589132786


In [26]:
# Define a list to store the data

os.getcwd()
os.listdir("C:\Thesis\Images")


['3M_2016', 'AaltoUniversity_2015']

In [27]:
pdf_directory = os.listdir("C:\Thesis\Images")

print(pdf_directory)

['3M_2016', 'AaltoUniversity_2015']


In [54]:
image_data = []

for foldername in pdf_directory:
    #print(foldername)
    image_directory = os.listdir("C:\\Thesis\\Images\\" + foldername)
    
    for filename_withEXT in image_directory:
        #print(foldername + filename_withEXT)
        img_path = os.path.join("C:\\Thesis\\Images",foldername, filename_withEXT)
      
        try:
            green_perc = extract_green_colors(img_path)
            blue_perc = extract_blue_colors(img_path)
            red_perc = extract_red_colors(img_path)
            live_obj = detect_human(img_path)
            #dominant_color = find_dominant_color(image_path)
            #human_obj =detect_human_yolo(image_path)
            
            probs_list = image_object_classification(img_path)
            probabilities = probs_list[0]
            probs = probabilities[0]
            #print(probs_list)
            is_human = probs[0]
            is_animal = probs[1]
            is_nature = probs[2]
            is_vehicle_or_machine = probs[3]
            is_building = probs[4]
            
           
            #image_data.append([foldername+".pdf", filename_withEXT, green_perc, live_obj])
            #print(img_path)
            
            image_data.append({
                'File Name': foldername + ".pdf",
                'Image Name': filename_withEXT,
                'green_perc': green_perc,
                'blue_perc': blue_perc,
                'red_perc': red_perc,
                'live_obj': live_obj,
                'human_obj_prob': is_human,
                'animal_obj_prob': is_animal,
                'nature_obj_prob': is_nature,
                'vehicleMachine_obj_prob': is_vehicle_or_machine,
                'is_building': is_building
            })
        
        except:
            print("Currupted File"+filename_withEXT)
       




`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["bos_token_id"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["eos_token_id"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["bos_token_id"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["eos_token_id"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is pro

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["eos_token_id"]` will be overriden.


In [55]:
df_image_date = pd.DataFrame(image_data)
df_image_date.head()

,File Name,Image Name,green_perc,blue_perc,red_perc,live_obj,human_obj_prob,animal_obj_prob,nature_obj_prob,vehicleMachine_obj_prob,is_building
0,3M_2016.pdf,page_10_image_1.png,0.491557,0.000000,1.063112,False,0.175771,0.009935,0.183137,0.228317,0.402839
1,3M_2016.pdf,page_10_image_2.png,3.953517,0.000000,0.000000,False,0.003600,0.011896,0.011956,0.970780,0.001769
2,3M_2016.pdf,page_11_image_1.png,1.761329,4.315191,0.006373,True,0.583358,0.035766,0.006755,0.240592,0.133529
3,3M_2016.pdf,page_19_image_1.png,0.261296,30.138868,0.174595,True,0.581364,0.050511,0.025460,0.239830,0.102835
4,3M_2016.pdf,page_1_image_1.png,10.221913,30.685470,0.184290,True,0.463595,0.014668,0.432330,0.052380,0.037028


In [56]:
# Load the extracted data file
df = pd.read_csv("Savefiles/ExtractedData.csv")
df.head()

,File Name,Image Name,Page Number,Text Before,Text After,Text Inside Image
0,1time_2011.pdf,page_2_image_1.png,2,| highlights 1 | 5-year review 2 | directorate...,NaN,NaN
1,1time_2011.pdf,page_2_image_2.png,2,| highlights 1 | 5-year review 2 | directorate...,NaN,NaN
2,1time_2011.pdf,page_3_image_1.png,3,NaN,NaN,NaN
3,1time_2011.pdf,page_3_image_2.png,3,NaN,NaN,NaN
4,1time_2011.pdf,page_3_image_3.png,3,NaN,NaN,NaN


In [57]:
# Merge df and df_image_date based on 'File Name' and 'Image Name'
df_new = df.merge(df_image_date, on=['File Name', 'Image Name'], how='left')

In [58]:
print(df_new)

          File Name             Image Name  Page Number  \
0    1time_2011.pdf     page_2_image_1.png            2   
1    1time_2011.pdf     page_2_image_2.png            2   
2    1time_2011.pdf     page_3_image_1.png            3   
3    1time_2011.pdf     page_3_image_2.png            3   
4    1time_2011.pdf     page_3_image_3.png            3   
..              ...                    ...          ...   
164     3M_2016.pdf  page_171_image_10.png          171   
165     3M_2016.pdf  page_171_image_11.png          171   
166     3M_2016.pdf  page_171_image_12.png          171   
167     3M_2016.pdf  page_171_image_13.png          171   
168     3M_2016.pdf  page_171_image_14.png          171   

                                           Text Before  \
0    | highlights 1 | 5-year review 2 | directorate...   
1    | highlights 1 | 5-year review 2 | directorate...   
2                                                  NaN   
3                                                  NaN   
4

In [97]:
df_new.to_csv('SaveFiles/ExtractedData_withGreenAndLiveObj.csv', index=False)